In [160]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [161]:
import json
import http.client
import mimetypes

from datetime import datetime

import plotly.offline as py
from plotly.offline import init_notebook_mode
import plotly.graph_objs as go

import pandas as pd
import numpy as np

In [162]:
list_of_countries = ["CHN","BEL","NLD","ITA","ESP","USA"]

In [163]:

records = []

def get_covid_data_for_country(country_code):
    conn = http.client.HTTPSConnection("covidapi.info")
    payload = ''
    headers = {}
    conn.request("GET", f"/api/v1/country/{country_code}/timeseries/2020-01-01/2020-04-12", payload, headers)
    res = conn.getresponse()
    return json.loads(res.read())["result"]

for country_code in list_of_countries:
    country_recs = get_covid_data_for_country(country_code)
    for rec in country_recs:
        rec["country"] = country_code
    records.extend(country_recs)

#print(json.dumps(records, indent=2))

In [164]:
covid_df = pd.DataFrame(records, columns=["confirmed","date","deaths","recovered","country"])
print(covid_df)

     confirmed        date  deaths  recovered country
0          548  2020-01-22      17         28     CHN
1          643  2020-01-23      18         30     CHN
2          920  2020-01-24      26         36     CHN
3         1406  2020-01-25      42         39     CHN
4         2075  2020-01-26      56         49     CHN
..         ...         ...     ...        ...     ...
481     396223  2020-04-07   12722      21763     USA
482     429052  2020-04-08   14695      23559     USA
483     461437  2020-04-09   16478      25410     USA
484     496535  2020-04-10   18586      28790     USA
485     526396  2020-04-11   20463      31270     USA

[486 rows x 5 columns]


In [196]:
pivot_df = pd.pivot_table(covid_df, values='deaths', index=['date'],columns='country', aggfunc=np.sum)

In [197]:
diff_df = pivot_df.diff(periods=7)

In [198]:
def create_country_traces(df):
    traces = []
    for col in df.columns:
        traces.append(go.Scatter(x=df.index, y=df[col].values,
                             name = col,
                             mode = 'markers+lines',
                             line=dict(shape='linear'),
                             connectgaps=True
                             )
                 )
    return traces

In [199]:
fig = go.Figure()
for trace in create_country_traces(pivot_df):
    fig.add_trace(trace)
fig.update_layout(
    width=1920,
    height=1080
)
fig.show()

In [200]:
fig = go.Figure()
for trace in create_country_traces(pivot_df):
    fig.add_trace(trace)
fig.update_layout(
    yaxis_type="log",
    width=1920,
    height=1080
)
fig.show()

In [201]:
merged_df = pivot_df.merge(diff_df, left_index=True, right_index=True, suffixes=("_total","_difference"))

In [202]:
merged_df.columns

Index(['BEL_total', 'CHN_total', 'ESP_total', 'ITA_total', 'NLD_total',
       'USA_total', 'BEL_difference', 'CHN_difference', 'ESP_difference',
       'ITA_difference', 'NLD_difference', 'USA_difference'],
      dtype='object', name='country')

In [203]:
fig = go.Figure()
for country_code in list_of_countries:
    col_total = f"{country_code}_total"
    col_difference = f"{country_code}_difference"
    fig.add_trace(go.Scatter(x=merged_df[col_total], y=merged_df[col_difference],
                             name = country_code,
                             mode = 'markers+lines',
                             line=dict(shape='linear'),
                             connectgaps=True
                )
    )
fig.update_layout(
    yaxis_title="New deaths in last week (log scale)",
    xaxis_title="Total deaths (log scale)",
    xaxis_type="log",
    yaxis_type="log",
    width=1920,
    height=1080
)
fig.show()

In [204]:
covid_df

,confirmed,date,deaths,recovered,country
0,548,2020-01-22,17,28,CHN
1,643,2020-01-23,18,30,CHN
2,920,2020-01-24,26,36,CHN
3,1406,2020-01-25,42,39,CHN
4,2075,2020-01-26,56,49,CHN
...,...,...,...,...,...
481,396223,2020-04-07,12722,21763,USA
482,429052,2020-04-08,14695,23559,USA
483,461437,2020-04-09,16478,25410,USA
484,496535,2020-04-10,18586,28790,USA
